# Target Propagation - Tutorial 1
## Standard Target Propagation

A simple implementation of target propagation to confirm that it works.

In [ ]:
import initialize

In [ ]:
from torchvision.datasets import FashionMNIST
from torchvision import transforms

from tools.modules import Sign, Stochastic, Clamp
from tools.training import train, classify
from tools import training
from tools.learners.target_prop import TargetPropLearner, AlternateTraining
from tools.learners.target_prop import BaselineLearner1

# Steps

1) Create each layer (AutoencoderLearner)
2) Create the TargetPropLearner
3) Run the training on the baseline
4) Run the training on the target propagation learner 

In [ ]:
transform = transforms.Compose(
    [transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))])

training_dataset = FashionMNIST(
    '../../Datasets/',
    transform=transform, download=True
)

testing_dataset = FashionMNIST(
    '../../Datasets/', train=False,
    transform=transform, download=True
)



In [ ]:
learner = BaselineLearner1(
    784, 300, 300, 300, 10
)
baseline_loss = train(learner, training_dataset, 10, device='cpu')
classify(learner, testing_dataset)

# TargetPropLearner

In [ ]:
learner = TargetPropLearner(
    784, 300, 300, 300, 10, dropout_p=0.0, out_x_lr=None
)

alternator = AlternateTraining(learner, 4, 8)


target_loss = train(learner, training_dataset, 50, device='cpu', callback=alternator)

classify(learner, testing_dataset)

In [ ]:
from torch import nn

learner = TargetPropLearner(
    784, 300, 300, 300, 10, act=nn.Tanh, reverse_act=nn.Tanh, in_act=Sign, out_x_lr=1.0, use_norm=False
)


target_loss = train(learner, training_dataset, 50, device='cpu')

classify(learner, testing_dataset)

In [ ]:
from torch import nn
import torch

print('Create learner')


learner = TargetPropLearner(
    784, 300, 300, 300, 10, nn.Tanh, nn.Tanh, lambda: torch.sign, dropout_p=0.5, out_x_lr=None
)

alternator = AlternateTraining(learner, 1, 4)

hard_target_loss = train(learner, training_dataset, 20, device='cpu', callback=alternator)

classify(learner, testing_dataset)


In [ ]:
from torch import nn
import torch

print('Create learner')
learner = TargetPropLearner(
    784, 300, 300, 300, 10, nn.Sigmoid, nn.Sigmoid, Stochastic, dropout_p=0.25, out_x_lr=1.0
)

alternator = AlternateTraining(learner, 1, 4)

hard_target_loss = train(learner, training_dataset, 20, device='cpu')

classify(learner, testing_dataset)

In [ ]:
training.plot_loss_line(
    [baseline_loss, target_loss, hard_target_loss], 
    ['Baseline', 'Target Prop - Leaky ReLU', 'Target Prop - Sign'], 
    'Training Loss', save_file='images/t2x1_target_prop.png'
)

In [ ]:
import pickle

losses = {'target1': target_loss, 'target2': target_loss2}

with open('results/t2x1_loss_results1.pkl', 'wb') as file:
    pickle.dump(losses, file)